In [ ]:
!pip install transformers

In [ ]:
import pandas as pd

In [ ]:
'''
load your path of training and evaluation dataset after preprocessing step.
in this work, we use 90% of training dataset as train data, while 10% of training dataset used as evaluation data.
you can split dataset with stratified K-fold Cross validation, since the dataset is multiclass and imbalanced
This code could used for monolingual, multilingual, and zero-shot sentiment analysis
'''
train = pd.read_excel(train dataset path) 
eval = pd.read_excel(evaluation dataset path) 

In [ ]:
'''
map the string label into number label
'''

train["new label"] = train['label'].map({'negative': 0, 'neutral': 1, 'positive': 2})
eval["new label"] = eval['label'].map({'negative': 0, 'neutral': 1, 'positive': 2})

In [ ]:
train_texts = train["preprocess"].tolist()
val_texts = eval["preprocess"].tolist()
train_labels = train["new label"].tolist()
val_labels = eval["new label"].tolist()

In [ ]:
from transformers import AutoTokenizer, AutoConfig
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/SA/Afriberta Tokenizer/All', config=AutoConfig.from_pretrained('castorini/afriberta_large'))

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

In [ ]:
import torch

class AfriDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = AfriDataset(train_encodings, train_labels)
val_dataset = AfriDataset(val_encodings, val_labels)

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/SA/out',   # output directory
    num_train_epochs=1,                           # total number of training epochs
    per_device_train_batch_size=32,               # batch size per device during training
    per_device_eval_batch_size=32,                # batch size for evaluation
    logging_dir='/content/drive/MyDrive/SA/dir',  # directory for storing logs
    eval_steps=10,
    logging_steps = 10,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    save_steps=100
)

model = AutoModelForSequenceClassification.from_pretrained("castorini/afriberta_large", num_labels=3)

trainer = Trainer(
    model=model,                         
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=train_dataset,            

)

trainer.train()

In [ ]:
trainer.save_model(model path) #save the trained model to used later